## Stochastic Modeling 
### GW1 1

## Step 1

### 1a Calibrate Heston(1993) Model to Market Data Using Lewis Approach

Heston Model introduces **Stochastic volatility**, where the variance $v_t$ follows a mean-reverting square-root process:
$$
dS_t = rS_tdt+\sqrt{v_t}S_tdW_t^S \\
dv_t = \kappa_v(\theta_v-v_t)dt + \sigma_v\sqrt{v_t}W_t^v \\
dW_t^SdW_t^v=\rho dt
$$

Where:
- $\kappa_v$ : Mean-reversion speed of volatilitty
- $\theta_v$: Long-term volatility level.
- $\sigma_v$: Volatility of volatility
- $\rho$: Correlation between asset returns and volatility
- $v_0$: Initial volatility

What we requere to do the caliberation is the characteristic fucnito of Heston which is defined as:
$$
\varphi^{H93}(u, T) = \exp\left(H_1(u, T) + H_2(u, T) \cdot v_0\right)
$$  
Where:  
- $ H_1(u, T) = r u i T + \frac{c_1}{\sigma_v^2} \left[ (\kappa_v - \rho \sigma_v u i + c_2) T - 2 \ln\left( \frac{1 - c_3 e^{c_2 T}}{1 - c_3} \right) \right] $ 

- $ H_2(u, T) = \frac{\kappa_v - \rho \sigma_v u i + c_2}{\sigma_v^2} \cdot \left[ \frac{1 - e^{c_2 T}}{1 - c_3 e^{c_2 T}} \right] $  

- $ c_1 = \kappa_v \theta_v $
- $ c_2 = -\sqrt{(\rho \sigma_v u i - \kappa_v)^2 - \sigma_v^2(-u i - u^2)} $  

- $ c_3 = \frac{\kappa_v - \rho \sigma_v u i + c_2}{\kappa_v - \rho \sigma_v u i - c_2} $



For our python implemenataion, we will begin by defining the Heston Model functions as follows

In [17]:
import numpy as np
from scipy.integrate import quad
import pandas as pd
from scipy.optimize import brute, fmin
# Characteristic Function
def Heston93_char_func(u, T, r, kappa_v, theta_v, sigma_v, rho, v0):
    c1 = kappa_v * theta_v
    c2 = -np.sqrt((rho * sigma_v * u * 1j) ** 2 - sigma_v ** 2 * (-u * 1j - u ** 2))
    c3 = (kappa_v - rho * sigma_v * u + c2) / (kappa_v - rho * sigma_v * u - c2)

    H1 = r * u * 1j * T + (c1/sigma_v ** 2) * ((kappa_v * rho * sigma_v * u * 1j + c2) * T - 2 * np.log((1 - c3 * np.exp(c2 * T))/ (1 - c3)))
    H2 = ((kappa_v - rho * sigma_v * u * 1j + c2)/ sigma_v**2 * ((1 - np.exp(c2 * T) / (1 - c3 * np.exp(c2 * T)))))

    char_func_value = np.exp (H1 + H2 * v0)

    return char_func_value

# Lewis (2001) integral function
def Heston93_int_func(u, S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0):
    char_func_value = Heston93_char_func(u - 1j*0.5, T, r, kappa_v, theta_v, sigma_v, rho, v0)
    return (1 / (u**2 + 0.25)) * (np.exp(1j * u * np.log(S0 / K)) * char_func_value)

# Heston call option  pricing
def Heston93_call_value(S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0):
    int_val = quad(lambda u: Heston93_int_func(u, S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0), 0, np.inf, limit=250)[0]
    return max(0, S0 - np.exp(-r * T ) * np.sqrt(S0 * K) / np.pi * int_val)

# Heston put option via Put-Call parity
def Heston93_put_value(S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0):
    call_price = Heston93_call_value(S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0)
    return call_price - S0 + K * np.exp(-r * T)



Next we define the calibration error funciton

In [16]:
i = 0
min_MSE = 5000

def Heston93_error_function(p0, option_data, S0):
    global i, min_MSE
    kappa_v, theta_v, sigma_v, rho, v0 = p0

    # Parameter constraints
    if kappa_v < 0 or theta_v < 0.005 or sigma_v < 0 or rho < -1 or rho > 1:
        return 5000
    
    # Compute MSE
    se = []
    for _, option in option_data.iterrows():
        K = option_data["Strike"]
        T = option_data["T"]
        r = option_data["r"]
        if option_data["Type"] == 'C':
            model_value = Heston93_call_value(S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0)
        else:
            model_value = Heston93_put_value(S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0)
        se.append((model_value - option_data["Price"])**2)
    MSE = sum(se) / len(se)
    min_MSE = min(min_MSE, MSE)
    if i % 23 == 0:
        print(f"Iteration {i}: Parameters {p0}, MSE {MSE:.3f}, Min MSE {min_MSE:.3f}")
    i += 1
    return MSE



Before we  run the simulations and calibrations let's load the data from  the google  sheet provided


In [ ]:
sheet_id = "1YNqTHLMxoGpyehXfpNg7WMbDSdnYQPwR"
sheet_name = "1"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

option_data = pd.read_csv(url)

option_data.head()

,Days to maturity,Strike,Price,Type
0,15,227.5,10.52,C
1,15,230.0,10.05,C
2,15,232.5,7.75,C
3,15,235.0,6.01,C
4,15,237.5,4.75,C
5,60,227.5,16.78,C
6,60,230.0,17.65,C
7,60,232.5,16.86,C
8,60,235.0,16.05,C
9,60,237.5,15.10,C


Let's enscaplute the simulation in a function  

In [ ]:
def Heston93_calibration():
    # Brute-froce scan ofr initial guesses
    p0 = brute(
        Heston93_error_function,
        (
            (0.5, 10.0, 2.0),    # kappa_v
            (0.01, 0.041, 0.01), # theta_v
            (0.05, 0.3, 0.1),    # sigma_v
            (-0.9, 0.0, 0.2),    # rho
            (0.01, 0.03, 0.01)   # v0
        ),
        finish=None
    )
    # Local optimization
    opt = fmin(Heston93_error_function, p0, xtol=0.0001, ftol=0.0001, maxiter=500, maxfun=1000)
    return opt